# Исследование надежности заемщиков.

## Цель исследования

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  

Входные данные от банка — статистика о платёжеспособности клиентов.  

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**

* `children` — количество детей в семье  
* `days_employed` — общий трудовой стаж в днях  
* `dob_years` — возраст клиента в годах  
* `education` — уровень образования клиента  
* `education_id` — идентификатор уровня образования  
* `family_status` — семейное положение  
* `family_status_id` — идентификатор семейного положения  
* `gender` — пол клиента  
* `income_type` — тип занятости  
* `debt` — имел ли задолженность по возврату кредитов  
* `total_income` — ежемесячный доходpurpose — цель получения кредита


## Исследование

### Шаг 2.0 Обзор данных

In [1]:
import pandas as pd
df = pd.read_csv('C://Users//u//Desktop//Projects//checking_reliability_borrowers//data_solvency.csv')

Воспользуемся методом .head() для обзора таблицы:

In [2]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Воспользуемся методом .info() для получения общей информации о таблице:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Судя по общей информации о таблице в столбцах "days_employed" и "total_income" есть пропущенные значения.

### Шаг 2.1 Заполнение пропусков

Согласно заданию на данном этапе не рассматривается столбец "days_employed".

Опредим какие значения находятся в ячейках с пропусками в столбце "total_income. Для этого выведем на экран 5 первых строк с пропущенными значениями:

In [4]:
df[df['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


В столбце "total_income" ячейки без значения имеют значеня NaN. 

NaN соответсвует типу данных float и это значит, что с ними можно проводить математические операции.

Проверим какую долю составляют пропущенные значения в каждом из столбцов с пропусками:

In [5]:
df['total_income'].isna().sum()/df.shape[0]

0.10099883855981417

In [6]:
df['days_employed'].isna().sum()/df.shape[0]

0.10099883855981417

Пропущенные значения в столбце "total_income" составлют 10% от всех значений столбце "total_income".

Пропущенные значения в столбце "days_employed" составлют 10% от всех значений столбце "days_employed".

Думаю, что примерно 10% клиентов работают неофициально и не могут подтвердит свой стаж и доход. Поэтому у таких 10% клиентов нет информации о трудовом стаже и ежемесячном доходе.

Пропущенные значения в столбце "total_income" лучше заменить на медианное значение, а не на среднее. Потому что среднее значение некорректно характиризует данные, если есть значения сильно отличающиеся от остальных. 

В столбце "total_income" заменим пропущенные значения на медианное значение:

In [7]:
total_income_median = df['total_income'].median()

In [8]:
df['total_income'] = df['total_income'].fillna(total_income_median)

Проверим сработала ли замена:

In [9]:
df['total_income'].isna().sum()

0

В столбце "total_income" не осталось пустых строк, значит замена сработала.

### Шаг 2.2 Проверка данных на аномалии и исправления.

При первом взгляде на столбец "days_employed" бросаются в глаза отрицательные значения.  
Для столбца с информацией об общем трудовом стаже в днях это странно.

Найдем кол-во таких строк:

In [10]:
df[df['days_employed'] < 0]['days_employed'].count()

15906

Доля от всех строк составляет:

In [11]:
df[df['days_employed'] < 0]['days_employed'].count()/df.shape[0]

0.7389547038327526

74% строк в столбце "days_employed" имеют отрицательные значения. Это достаточно много.

Предполагаю, что такое могло произойти из-за некорректно составленной анкеты для потенциального заёмщика. Из-за этого информацию о стаже заёмщик указывал через дефис.

Заменим отрицательные значения в столбце "days_employed" на такие же положительные значения:

In [12]:
df.loc[df['days_employed'] < 0, 'days_employed'] = abs(df['days_employed'])

Проверим корректно ли сработал код:

In [13]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Первые 4 значения в столбце "days_employed" были отрицательные. Теперь они стали положительными. Код сработал корректно.

In [14]:
df[df['days_employed'] < 0]['days_employed'].count()

0

В столбце "days_employed" больше нет отрицательных значений.

Заменим пропущение значения в столбце "days_employed" на медианное значение:

In [15]:
days_employed_median = df['days_employed'].median()

In [16]:
df['days_employed']  = df['days_employed'].fillna(days_employed_median)

Проверим сработала ли замена:

In [17]:
df['days_employed'].isna().sum()

0

В столбце "days_employed" не осталось пустых строк, значит замена сработала.

### Шаг 2.3. Изменение типов данных.

Соласно заданию изменим тип данных столбца "total_income" на целочисленный

In [18]:
df['total_income'] = df['total_income'].astype('int')

### Шаг 2.4. Удаление дубликатов.

Проверим есть ли явные дубликаты в данных:

In [19]:
df.duplicated().sum()

54

Избавимся от явных дубликатов:

In [20]:
df = df.drop_duplicates().reset_index(drop=True)

In [21]:
df.duplicated().sum()

0

Выявим неявные дубликаты значений в столбцах, где это возможно:

In [22]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [23]:
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [24]:
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

In [25]:
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [26]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Видно, что в столбце "family_status" в значении "Не женат / не замужем" используется разный регистр.  
Исправим это:

In [27]:
df['family_status'] = df['family_status'].str.lower()

Видно, что в столбце "education" одни и те же значения записаны разным регистром.  
Исправим это:

In [28]:
df['education'] = df['education'].str.lower()

In [29]:
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Удалим явные дубликаты снова

In [30]:
df = df.drop_duplicates().reset_index(drop=True)

При выборе метода поиска дубликатов руководствовался целью.  
Если нужно найти явные дубликаты строк, то использовал df.duplicated().  
Если нужно найти неявные дубликаты в значениях, связанные, например, с разными регистрами, то использовал .value_counts()

Возможные причины возникновения дубликатов вижу следующие:  
`-` создание базы на основе данных из разных источник, что могло привести в задвоеннию информации;  
`-` некорретное заполнение анкеты;  
`-` некорретная анкета.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [31]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Создадим словарь для "education":

In [32]:
education_dict = df[['education', 'education_id']]

In [33]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

In [34]:
education_dict.sort_values('education_id')

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Создадим словарь для "family_status":

In [35]:
family_status_dict = df[['family_status', 'family_status_id']]

In [36]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

In [37]:
family_status_dict.sort_values('family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Удалим столбцы 'education' и 'family_status' из датафрейма:

In [38]:
df = df.drop('education', 1)

C:\Users\u\AppData\Local\Temp/ipykernel_7624/3333381138.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('education', 1)


In [39]:
df = df.drop('family_status', 1)

C:\Users\u\AppData\Local\Temp/ipykernel_7624/894069686.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('family_status', 1)


### Шаг 2.6. Категоризация дохода.

Напишем функцию для категоризации доходов:

In [40]:
def total_income_group(total_income):
    try:
        if 0 < total_income <= 30000:
            return 'E'
        if 30001 < total_income <= 50000:
            return 'D'
        if 50001 < total_income <= 200000:
            return 'C'
        if 200001 < total_income <= 1000000:
            return 'B'
        if total_income > 1000001:
            return 'A'
    except:
        return 'проверьте значение в столбце total_income' 

Добавим столбец с категориями по доходам применив функцию к столбцу "total_income":

In [41]:
df['total_income_category'] = df['total_income'].apply(total_income_group)

Создадим "словарь":

In [42]:
data_total_income_dict = [
    ['E', 'меньше 30000'],
    ['D', 'от 30000 до 50000'],
    ['С', 'от 50000 до 200000'],
    ['B', 'от 200000 до 1000000'],
    ['A', 'больше 1000000']
]
columns_total_income_dict = ['income_group', 'total_income']
total_income_dict = pd.DataFrame(data=data_total_income_dict, columns=columns_total_income_dict)

### Шаг 2.7. Категоризация целей кредита.

Выведем и проанализируем уникальные значения в столбце "purpose":

In [43]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Выберем характерные для каждой категории подстроки и создадим функцию:

In [44]:
def purpose_group(purpose):
    try:
        if 'авто' in purpose:
            return 'операции с автомобилем'
        if ('жиль' in purpose or
            'недвиж' in purpose):
            return 'операции с недвижимостью'
        if 'свадь' in purpose:
            return 'проведение свадьбы'
        if 'образов' in purpose:
            return 'получение образования'
    except:
        return 'другое'

Добавим столбец с категориями по целям кредита применив функцию к столбцу "purpose":

In [45]:
df['purpose_category'] = df['purpose'].apply(purpose_group)

In [46]:
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


<div class="alert alert-success">
<b>✔️ Комментарий ревьюера:</b> Категоризация выполнена корректно

### Ответы на вопросы.

При подготовке ответов в столбце "children" выявились некоторые анамалии, которые пришлось исправить:

In [47]:
df['children'] = abs(df['children'])

In [48]:
df['children'] = df['children'].replace(20, 2)

Для ответов на вопросы найдем какую долю состаляют люди, которые имели просрочки по кредиту от людей, которые просрочки ранее не имели.  
И далее сгрупируем эти данные по критерию, от которого нужно проверить зависимость.

Сформируем два вспомогательных датафрейма: клиенты без просрочек и клиенты с просрочками.

In [49]:
debt_0 = df[df['debt'] == 0]

In [50]:
debt_1 = df[df['debt'] == 1]

Напишем функцию, которая возвращает нам знаение в зависимости от критерия:

In [51]:
def ratio_debt(column):
    debt_column = debt_1.groupby(column)[column].count()/debt_0.groupby(column)[column].count()
    sorted_debt_column = debt_column.sort_values()
    return sorted_debt_column.head(10)

#### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для проверки зависимости между количеством детей и возвратом кредита в срок применим функцию к столбцу "children"

In [52]:
ratio_debt('children')

children
0    0.081593
3    0.089109
1    0.100907
2    0.104881
4    0.108108
5         NaN
Name: children, dtype: float64

#### Вывод 1:

Так как число заемщиков с 5 детьми мало по сравнению с другими заемщиками, то при составлении вывода данными по ним пренебрегаем.

Согласно результам доля имеющих просрочки клиентов от числа клиентов не имеющих просрочек составляет около 10%.  
При этом семьи без детей и семьи с 3 детьми имеют меньшую вероятность просрочить платеж.  
Можно сделать вывод, что нет прямой зависимости кол-ва детей в семье и возврата кредита в срок, но семьи без детей и семьи с 3 детьми более надежны, чем остальные. 

#### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для проверки зависимости между семейным положением и возвратом кредита в срок применим функцию к столбцу "family_status_id"

In [53]:
ratio_debt('family_status_id')

family_status_id
2    0.070312
3    0.076577
0    0.081609
1    0.103109
4    0.108044
Name: family_status_id, dtype: float64

Для расшифровки результатов выведем соответвующий "словарь"

In [54]:
family_status_dict.sort_values('family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


#### Вывод 2:

Согласно результам доля имеющих просрочки клиентов от числа клиентов не имеющих просрочек составляет от 7 до 10 %.  

Можно сделать вывод, что возврат кредита в срок зависит от семейного положения. Наиболее надежными являются люди в разводе или люди, потерявшие супруга или супругу. Менее надежными являются люди, состоящие в гражданском браке, или не состоящие в браке вообще. Люди, состоящие в браке, имеют среднюю надежность по сравнению с другими.

#### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для проверки зависимости между уровнем дохода и возвратом кредита в срок применим функцию к столбцу "total_income_category"

In [55]:
ratio_debt('total_income_category')

total_income_category
D    0.063830
B    0.076003
A    0.086957
C    0.092795
E    0.100000
Name: total_income_category, dtype: float64

Для расшифровки результатов выведем соответвующий "словарь"

In [56]:
total_income_dict.head()

,income_group,total_income
0,E,меньше 30000
1,D,от 30000 до 50000
2,С,от 50000 до 200000
3,B,от 200000 до 1000000
4,A,больше 1000000


#### Вывод 3: 

Согласно результам доля имеющих просрочки клиентов от числа клиентов не имеющих просрочек составляет от 6 до 10 %.  

Можно сделать вывод, что уровень дохода влияет на возврат кредита в срок зависит.  
Наиболее надежными являются люди с доходами категории D (от 30000 до 50000 р. в месяц) и категории B (от 200000 до 1000000 р. месяц).  
Менее надежными являются люди с доходами категории A (больше 1000000 р. в месяц) и категории C (от 50000 до 200000 р. в месяц).  
Самые ненадежные люди с доходами категории E (меньше 30000 р. в месяц).

#### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Для проверки зависимости между целью кредита и возвратом кредита в срок применим функцию к столбцу "purpose_category"

In [57]:
ratio_debt('purpose_category')

purpose_category
операции с недвижимостью    0.077974
проведение свадьбы          0.086997
получение образования       0.101565
операции с автомобилем      0.103254
Name: purpose_category, dtype: float64

#### Вывод 4: 

Согласно результам доля имеющих просрочки клиентов от числа клиентов не имеющих просрочек составляет от 8 до 10 %.  

Можно сделать вывод, что цель кредита влияет на возврат кредита в срок зависит.  
Наиболее надежными являются займы на операции с недвижимость и проведение свадьб.  
Менее надежными являются займы на образование и на операции с автомобилями.

## Общий вывод:

По результатам проведенного анализа можно сделать вывод, что семейное положение и количество детей клиента влияют на факт погашения кредита в срок.  

Нет прямой зависимости кол-ва детей в семье и возврата кредита в срок, но семьи без детей и семьи с 3 детьми более надежны, чем остальные.  

Наиболее надежными являются люди в разводе или люди, потерявшие супруга или супругу. Менее наджеными являются люди, состоящие в гражданском браке, или не состоящие в браке вообще. Люди, состоящие в браке, имеют среднюю надежность по сравнению с другими.

Кроме этого выяснилось, что на возврат кредита в срок также влияют уровень дохода клиента и цель кредита.

Наиболее надежными являются люди с доходами от 30000 до 50000 р. в месяц и доходами от 200000 до 1000000 р. месяц.  
Менее надежными являются люди с доходами больше 1000000 р. в месяц и доходами от 50000 до 200000 р. в месяц.  
Самые ненадежные люди с доходами категории меньше 30000 р. в месяц

Наиболее надежными являются займы на операции с недвижимость и проведение свадьб.  
Менее надежными являются займы на образование и на операции с автомобилями.

Можно сказать, что при построении модели кредитного скоринга неоходимо учитывать следующие критерии потенциального заемщика:
 * семейное положение;
 * количество детей в семье;
 * месячный уровень дохода;
 * цель кредита.